In [ ]:
import pyvista as pv
import glob
import os
import re

edpfile="ex_2_fra_corretto.edp"
command="FreeFem++ "
string=command+edpfile
print(f"Esecuzione di: {string}")
os.system(string)

Esecuzione di: FreeFem++ ex_2_fra_corretto.edp


0

Read and order VTK files. 

In [ ]:
# File selection and sorting
# Natural sort (1, 2, 10 instead of 1, 10, 2)
def natural_sort(l): 
    return sorted(l, key=lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else 0)

vtk_files = natural_sort(glob.glob("bolt_hydrogen_*.vtk"))

if not vtk_files:
    raise RuntimeError("No file bolt_hydrogen_*.vtk found!")
print(f"Found {len(vtk_files)} VTK files")

Trovati 11 file VTK


Let's try to plot the slices every timestep. We first create a loop for the time, then a loop for the slices

In [ ]:
zs = [0.08, 0.12, 0.25, 3]  # slices inside the head and the shank
cs_value = 10e-5            # as in the .edp file 
pv.set_jupyter_backend("static")

for i in range(0, len(vtk_files)):
    fname = vtk_files[i]
    print(f"Plot timestep {i}: {fname}")

    mesh = pv.read(fname)

    for z in zs:
        # Mesh slicing
        slc = mesh.slice(normal='z', origin=(0, 0, z))
        
        # If slice intersects the mesh: plot
        if slc.n_points > 0:
            p = pv.Plotter()
            # ADDED: force custom range
            p.add_mesh(slc, 
                       scalars="concentration", 
                       cmap="turbo", 
                       clim=[0, cs_value]) 
            
            # ADDED: Show borders (physical boundaries)
            p.add_mesh(slc.extract_feature_edges(), color="white", opacity=0.0, line_width=2)
            
            p.add_text(f"File: {fname}, z = {z}", font_size=10)
            p.view_xy() # Frontal 2D view
            p.screenshot(f"{fname}_{z}.png", transparent_background= True) 

Plot timestep 0: bolt_hydrogen_0.vtk
Plot timestep 1: bolt_hydrogen_1.vtk
Plot timestep 2: bolt_hydrogen_2.vtk
Plot timestep 3: bolt_hydrogen_3.vtk
Plot timestep 4: bolt_hydrogen_4.vtk
Plot timestep 5: bolt_hydrogen_5.vtk
Plot timestep 6: bolt_hydrogen_6.vtk
Plot timestep 7: bolt_hydrogen_7.vtk
Plot timestep 8: bolt_hydrogen_8.vtk
Plot timestep 9: bolt_hydrogen_9.vtk
Plot timestep 10: bolt_hydrogen_10.vtk


example in the file time_dependence.ipynb (DO NOT RUN)

In [ ]:
# scalar_name = 'u'   # dataname used in FreeFem++
# pv.set_jupyter_backend("static")  # try to use "trame" 

# step = 10 # Plot every 10 steps

# for i in range(0, len(vtk_files), step):
#     fname = vtk_files[i]
#     print(f"Plot timestep {i}: {fname}")

#     mesh = pv.read(fname)

#     p = pv.Plotter(notebook=True)

# # This is volume visualization, not a slice.
#     p.add_mesh(
#         mesh,
#         scalars=scalar_name,
#         show_edges=False,
#     )
#     p.add_scalar_bar(title=scalar_name)

# # Set the observation point for 2D problems
#     try:
#         p.camera_position = "xy"  # Used for 2D problems
#     except:
#         pass
#     p.show()